<a href="https://colab.research.google.com/github/Suparni-Maitra/test/blob/main/Submission_M25CSA017_M25CSA021_M25CSA029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFORMED SEARCH

## Puzzle Formulation Basic Functions

In [ ]:
class Puzzle:
    def __init__(self):
      pass

    def StringtoMatrix(self, s):
        #this converts the string to matrix form
        new_s=s.replace('B','0')
        rows=new_s.split(';')
        return [[int(ch) for ch in row] for row in rows]


    def blank_position(self, state):
        #this shows the position of the blank tile represented as 0
        for i in range(3):
            for j in range(3):
                if state[i][j]==0:
                    return (i, j)


    def manhattan_distance(self,state, goal_state):
      distance = 0
      for i in range(3):
          for j in range(3):
              if state[i][j] != 0:  # Don't count blank tile
                  value = state[i][j]
                  # Find target position of this value
                  target_i, target_j = None, None
                  for ti in range(3):
                      for tj in range(3):
                          if goal_state[ti][tj] == value:
                              target_i, target_j = ti, tj
                              break
                      if target_i is not None:
                          break
                  if target_i is not None:
                      distance += abs(i - target_i) + abs(j - target_j)
      return distance

    def misplaced_tiles(self, state, goal_state):
      misplaced = 0
      for i in range(3):
          for j in range(3):
              if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                  misplaced += 1
      return misplaced

    def goal_reached(self, state, goal_state):
        #this checks if the cuurent matrix is same as the goal matrix
        return state==goal_state

    def state_to_tuple(self, state):
      return tuple(tuple(row) for row in state)

    def get_neigh(self, state):
        #slides the blank tile around and finds the next states
        i, j=self.blank_position(state)
        moves=[]
        directions=[(-1,0),(1,0),(0,-1),(0,1)]
        for di, dj in directions:
            ni, nj=i+di, j+dj
            if 0<=ni<3 and 0<=nj < 3:
                new_state=[row[:] for row in state]
                new_state[i][j], new_state[ni][nj]=new_state[ni][nj], new_state[i][j]
                moves.append(new_state)
        return moves



In [ ]:
def read_input_file(filename):
  with open(filename, 'r') as file:
    content = file.read().strip()
  if content.startswith('{'):
    import json
    data = json.loads(content)
    start_string = data['start']
    goal_string = data['goal']
  else:
    lines = content.split('\n')
    start_string = lines[0].strip()
    goal_string = lines[1].strip()

  start_state = puzzle.StringtoMatrix(start_string)
  goal_state = puzzle.StringtoMatrix(goal_string)
  return start_state, goal_state

## Greedy Best First Search

In [ ]:
import heapq
puzzle = Puzzle()

In [ ]:
def greedy_best_first_search(start_state, goal_state, heuristic='manhattan'):
  if heuristic == 'manhattan':
        h_func = lambda state: puzzle.manhattan_distance(state, goal_state)
  elif heuristic == 'misplaced':
        h_func = lambda state: puzzle.misplaced_tiles(state, goal_state)

  heap = [(h_func(start_state), start_state, [])]
  visited = set()
  nodes_explored = 0

  while heap:
      h_value, current_state, path = heapq.heappop(heap)
      state_tuple = puzzle.state_to_tuple(current_state)

      if state_tuple in visited:
          continue

      visited.add(state_tuple)
      nodes_explored += 1

      # Check if goal reached
      if puzzle.goal_reached(current_state, goal_state):
          return {'heuristic_used': heuristic, 'path': path, 'nodes_explored': nodes_explored,'path_cost': len(path)}

      # Explore neighbors
      for next_state in puzzle.get_neigh(current_state):
          next_state_tuple = puzzle.state_to_tuple(next_state)
          if next_state_tuple not in visited:
              h_next = h_func(next_state)
              heapq.heappush(heap, (h_next, next_state, path + [next_state]))


In [ ]:
read_input_file('input.txt')

([[1, 2, 3], [0, 4, 6], [7, 5, 8]], [[1, 2, 3], [4, 5, 6], [7, 8, 0]])

IMPLEMENTATION OF THE FUNCTIONS

In [ ]:
greedy_start_state, greedy_end_state = read_input_file('input.txt')

In [ ]:
greedy_best_first_search(greedy_start_state,greedy_end_state,'manhattan')

{'heuristic_used': 'manhattan',
 'path': [[[1, 2, 3], [4, 0, 6], [7, 5, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 0, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]],
 'nodes_explored': 4,
 'path_cost': 3}

In [ ]:
greedy_best_first_search(greedy_start_state,greedy_end_state,'misplaced')

{'heuristic_used': 'misplaced',
 'path': [[[1, 2, 3], [4, 0, 6], [7, 5, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 0, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]],
 'nodes_explored': 4,
 'path_cost': 3}

## A* Search

In [ ]:
def a_star_search(start_state, goal_state, heuristic='manhattan'):
  if heuristic == 'manhattan':
        h_func = lambda state: puzzle.manhattan_distance(state, goal_state)
  elif heuristic == 'misplaced':
        h_func = lambda state: puzzle.misplaced_tiles(state, goal_state)

  heap = [(h_func(start_state), 0, start_state, [])]
  visited = set()
  nodes_explored = 0

  while heap:
      f_value, g_value, current_state, path = heapq.heappop(heap)
      state_tuple = puzzle.state_to_tuple(current_state)

      if state_tuple in visited:
          continue

      visited.add(state_tuple)
      nodes_explored += 1

      # Check if goal reached
      if puzzle.goal_reached(current_state, goal_state):
          return {'heuristic_used': heuristic, 'path': path, 'nodes_explored': nodes_explored, 'path_cost': len(path)}

      # Explore neighbors
      for next_state in puzzle.get_neigh(current_state):
          next_state_tuple = puzzle.state_to_tuple(next_state)
          if next_state_tuple not in visited:
              g_next = g_value + 1  # Cost to reach next state
              h_next = h_func(next_state)
              f_next = g_next + h_next  # Total cost estimate
              heapq.heappush(heap, (f_next, g_next, next_state, path + [next_state]))

IMPLEMENTATION OF THE FUNCTIONS

In [ ]:
a_start_state, a_end_state = read_input_file('input.txt')

In [ ]:
a_star_search(a_start_state,a_end_state,'manhattan')

{'heuristic_used': 'manhattan',
 'path': [[[1, 2, 3], [4, 0, 6], [7, 5, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 0, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]],
 'nodes_explored': 4,
 'path_cost': 3}

In [ ]:
a_star_search(a_start_state,a_end_state,'misplaced')

{'heuristic_used': 'misplaced',
 'path': [[[1, 2, 3], [4, 0, 6], [7, 5, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 0, 8]],
  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]],
 'nodes_explored': 4,
 'path_cost': 3}

# MEMORY BOUNDED SEARCH

## Iterative Deepening A* (IDA*)

In [ ]:
def ida_star_search(start_state, goal_state, heuristic='manhattan'):
  if heuristic == 'manhattan':
        h_func = lambda state: puzzle.manhattan_distance(state, goal_state)
  elif heuristic == 'misplaced':
        h_func = lambda state: puzzle.misplaced_tiles(state, goal_state)

  threshold = h_func(start_state)
  total_nodes_explored = 0

  while True:
        result, new_threshold, nodes_explored = _ida_search_recursive(
            start_state, goal_state, 0, threshold, set(), h_func
        )
        total_nodes_explored += nodes_explored

        if result['success']:
            result['nodes_explored'] = total_nodes_explored
            result['heuristic_used'] = heuristic
            return result

        if new_threshold == float('inf'):
            return {
                'heuristic_used': heuristic,
                'nodes_explored': total_nodes_explored,
            }

        threshold = new_threshold

In [ ]:
def _ida_search_recursive(state, goal_state, g, threshold, visited, h_func):
    f = g + h_func(state)

    if f > threshold:
        return {'success': False}, f, 1

    if puzzle.goal_reached(state, goal_state):
        return {'success': True}, 0, 1

    state_tuple = puzzle.state_to_tuple(state)
    visited.add(state_tuple)

    min_threshold = float('inf')
    nodes_explored = 1

    for next_state in puzzle.get_neigh(state):
        next_state_tuple = puzzle.state_to_tuple(next_state)
        if next_state_tuple not in visited:
            result, new_threshold, explored = _ida_search_recursive(
                next_state, goal_state, g + 1, threshold, visited.copy(), h_func
            )
            nodes_explored += explored

            if result['success']:
                return result, 0, nodes_explored

            if new_threshold < min_threshold:
                min_threshold = new_threshold

    return {'success': False}, min_threshold, nodes_explored

IMPLEMETATION OF FUNCTIONS

In [ ]:
ida_start_state, ida_end_state = read_input_file('input.txt')

In [ ]:
ida_star_search(ida_start_state,ida_end_state,'manhattan')

{'success': True, 'nodes_explored': 8, 'heuristic_used': 'manhattan'}

In [ ]:
ida_star_search(ida_start_state,ida_end_state,'misplaced')

{'success': True, 'nodes_explored': 8, 'heuristic_used': 'misplaced'}

# UNINFORMED SEARCH

## BFS

Load Input

In [ ]:
import json
from collections import deque
import time

def read_input(file_path="input.txt"):
    with open(file_path, "r") as f:
        data = json.load(f)
    start = data["start"].replace(";", "")
    goal = data["goal"].replace(";", "")
    return start, goal

def get_neighbors(state):
    neighbors = []
    moves = [(-1,0),(1,0),(0,-1),(0,1)]
    index = state.index("B")
    x, y = divmod(index, 3)

    for dx, dy in moves:
        nx, ny = x+dx, y+dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_index = nx*3 + ny
            state_list = list(state)
            state_list[index], state_list[new_index] = state_list[new_index], state_list[index]
            neighbors.append("".join(state_list))
    return neighbors

def reconstruct_path(parents, state):
    path = []
    while state in parents:
        path.append(state)
        state = parents[state]
    path.reverse()
    return path

def format_state(state):
    return state[0:3] + ";" + state[3:6] + ";" + state[6:9]

In [ ]:
def bfs(start, goal):
    start_time = time.time()
    queue = deque([start])
    visited = set([start])
    parents = {}
    nodes_explored = 0

    while queue:
        state = queue.popleft()
        nodes_explored += 1

        if state == goal:
            path = reconstruct_path(parents, state)
            elapsed = time.time() - start_time
            return True, path, visited, nodes_explored, elapsed

        for neighbor in get_neighbors(state):
            if neighbor not in visited:
                visited.add(neighbor)
                parents[neighbor] = state
                queue.append(neighbor)

    elapsed = time.time() - start_time
    return False, [], visited, nodes_explored, elapsed

## DFS

In [ ]:
def dfs(start, goal, depth_limit=50):
    start_time = time.time()
    stack = [start]
    visited = set([start])
    parents = {}
    nodes_explored = 0

    while stack:
        state = stack.pop()
        nodes_explored += 1

        if state == goal:
            path = reconstruct_path(parents, state)
            elapsed = time.time() - start_time
            return True, path, visited, nodes_explored, elapsed

        if len(reconstruct_path(parents, state)) < depth_limit:
            for neighbor in get_neighbors(state):
                if neighbor not in visited:
                    visited.add(neighbor)
                    parents[neighbor] = state
                    stack.append(neighbor)

    elapsed = time.time() - start_time
    return False, [], visited, nodes_explored, elapsed

Print BFS and DFS

In [ ]:
def write_output(success, path, visited, nodes_explored, elapsed_time,
                 algo_name, start, goal):
    print(f"\n{algo_name}")
    if success:
        print(f"Step 0: {format_state(start)}")
        for i, state in enumerate(path, 1):
            print(f"Step {i}: {format_state(state)}")
        print(f"Visited: {len(visited)} | Nodes Explored: {nodes_explored} | Time: {elapsed_time:.4f} s")
    else:
        print("No solution found.")
        print(f"Visited: {len(visited)} | Nodes Explored: {nodes_explored} | Time: {elapsed_time:.4f} s")

In [ ]:
start, goal = read_input("input.txt")

bfs_result = bfs(start, goal)
write_output(*bfs_result, "BFS", start, goal)

dfs_result = dfs(start, goal)
write_output(*dfs_result, "DFS", start, goal)



BFS
Step 0: 123;B46;758
Step 1: 123;4B6;758
Step 2: 123;456;7B8
Step 3: 123;456;78B
Visited: 30 | Nodes Explored: 17 | Time: 0.0001 s

DFS
Step 0: 123;B46;758
Step 1: 123;4B6;758
Step 2: 123;46B;758
Step 3: 123;468;75B
Step 4: 123;468;7B5
Step 5: 123;4B8;765
Step 6: 123;48B;765
Step 7: 123;485;76B
Step 8: 123;485;7B6
Step 9: 123;4B5;786
Step 10: 123;45B;786
Step 11: 123;456;78B
Visited: 107118 | Nodes Explored: 107110 | Time: 0.6688 s


#LOCAL/ EVOLUTIONARY SEARCH ALGORITHMS

In [ ]:
#@title Helper Functions for hill climbing, simulated annealing and genetic algorithms

import random, math, time, json, os
random.seed(42)

def string_to_matrix(s):
    #Converts '123;B46;758' into a 3x3 matrix with 0 for blank.
    new_s=s.replace('B','0')
    return [[int(ch) for ch in row] for row in new_s.split(';')]

def matrix_to_string(matrix):
    #Convert 3x3 matrix back to '123;456;78B'
    return ";".join("".join(str(x) if x!=0 else "B" for x in row) for row in matrix)

def blank_position(state):
    #Return (i,j) of the blank (0).
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return (i, j)

def find_neighbors(state):
    #Generate all valid neighbor states by sliding blank.
    i,j=blank_position(state)
    moves=[]
    for di,dj in [(-1,0),(1,0),(0,-1),(0,1)]:
        ni,nj=i+di,j+dj
        if 0<=ni<3 and 0<=nj<3:
            new_state=[row[:] for row in state]
            new_state[i][j], new_state[ni][nj]=new_state[ni][nj],new_state[i][j]
            moves.append(new_state)
    return moves

def h_misplaced(state, goal):
    #Heuristic: number of misplaced tiles.
    return sum(
        1 for i in range(3) for j in range(3)
        if state[i][j] != 0 and state[i][j] != goal[i][j]
    )

def h_manhattan(state, goal):
    #Heuristic: sum of Manhattan distances.
    pos = {goal[i][j]:(i,j) for i in range(3) for j in range(3)}
    dist = 0
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile != 0:
                gi, gj = pos[tile]
                dist += abs(i-gi) + abs(j-gj)
    return dist



In [ ]:
#@title Load data from input.txt
if os.path.exists("input.txt"):
    with open("input.txt") as f:
        cfg = json.load(f)
else:
    raise FileNotFoundError("Upload input.txt first!")

start=string_to_matrix(cfg["start"])
goal=string_to_matrix(cfg["goal"])

print("Start:", cfg["start"])
print("Goal :", cfg["goal"])



Start: 123;B46;758
Goal : 123;456;78B


##Hill Climbing

In [ ]:
def hill_climbing(start, goal, max_iters=50):
    cur=start
    visited=[cur]
    for _ in range(max_iters):
        if cur==goal: break
        neigh=find_neighbors(cur)
        best=min(neigh, key=lambda s: h_misplaced(s, goal))
        if h_misplaced(best, goal)>=h_misplaced(cur, goal):
            break
        cur=best
        visited.append(cur)
    return visited

## Simulated Annealing

In [ ]:
def simulated_annealing(start, goal, max_iters=300, T0=2.0):
    cur = start
    visited=[cur]
    T=T0
    for _ in range(max_iters):
        if cur==goal: break
        nxt=random.choice(find_neighbors(cur))
        d=h_manhattan(nxt, goal)-h_manhattan(cur, goal)
        if d<=0 or random.random()<math.exp(-d/T):
            cur=nxt
        if cur not in visited:
            visited.append(cur)
        T*=0.98
    return visited

## Genetic Algorithm (with Fitness Function)

In [ ]:
def fitness(state, goal):
    #Fitness function: higher for states closer to goal (Manhattan).
    return 1 / (1 + h_manhattan(state, goal))

def tournament_selection(pop, goal, k=3):
    #Select the fittest individual from k random candidates.
    selected=random.sample(pop, k)
    return max(selected, key=lambda s: fitness(s, goal))

def crossover(p1, p2):
    #take first few rows from p1, rest from p2
    point=random.randint(1, 2)  # crossover after row 1 or 2
    child=[row[:] for row in p1[:point]] + [row[:] for row in p2[point:]]
    return child

def mutate(state):
    #swap two random tiles (excluding blank)
    new_state = [row[:] for row in state]
    tiles = [(i, j) for i in range(3) for j in range(3) if new_state[i][j] != 0]
    a, b = random.sample(tiles, 2)
    new_state[a[0]][a[1]], new_state[b[0]][b[1]] = new_state[b[0]][b[1]], new_state[a[0]][a[1]]
    return new_state

def genetic_algorithm(start, goal, pop_size=10, generations=5,
                      mutation_rate=0.3, crossover_method="order",
                      selection_method="tournament"):
    # Initialize population
    pop = [start] + [random.choice(find_neighbors(start)) for _ in range(pop_size - 1)]
    visited = set(tuple(sum(p, [])) for p in pop)

    for _ in range(generations):
        # Rank population by fitness (higher is better)
        pop = sorted(pop, key=lambda s: fitness(s, goal), reverse=True)
        visited.update(tuple(sum(p, [])) for p in pop)
        if pop[0]==goal:
            break
        # Elitism: carry over top 5
        new_pop=pop[:5]
        while len(new_pop) < pop_size:
            # Selection
            if selection_method=="tournament":
                p1 = tournament_selection(pop, goal)
                p2 = tournament_selection(pop, goal)
            else:  # fallback: random selection
                p1, p2 = random.sample(pop, 2)
            # Crossover
            if crossover_method == "order":
                child = crossover(p1, p2)
            else:
                child = random.choice([p1, p2])  # fallback: cloning
            # Mutation
            if random.random() < mutation_rate:
                child=mutate(child)
            new_pop.append(child)
        pop = new_pop
    # Convert visited back to 3x3 matrix states
    visited_states = []
    for flat in visited:
        m = [list(flat[i:i+3]) for i in range(0, 9, 3)]
        visited_states.append(m)

    return visited_states


In [ ]:
#@title Run All Algorithms & Print States

def print_states(states, title):
    print(f"\n--- {title} ---")
    for i, s in enumerate(states):
        print(f"Step {i}: {matrix_to_string(s)}")

# Hill Climbing
t=time.time()
hc_path=hill_climbing(start, goal)
print_states(hc_path, "Hill Climbing")
print("Visited:", len(hc_path), "| Time:", round(time.time()-t, 3), "s")

# Simulated Annealing
t=time.time()
sa_path=simulated_annealing(start, goal)
print_states(sa_path, "Simulated Annealing")
print("Visited:", len(sa_path), "| Time:", round(time.time()-t, 3), "s")

# Genetic Algorithm
t=time.time()
ga_path = genetic_algorithm(start, goal)
print_states(ga_path, "Genetic Algorithm")
print("Visited:", len(ga_path), "| Time:", round(time.time()-t, 3), "s")




--- Hill Climbing ---
Step 0: 123;B46;758
Step 1: 123;4B6;758
Step 2: 123;456;7B8
Step 3: 123;456;78B
Visited: 4 | Time: 0.0 s

--- Simulated Annealing ---
Step 0: 123;B46;758
Step 1: 123;4B6;758
Step 2: 1B3;426;758
Step 3: B13;426;758
Step 4: 413;B26;758
Step 5: 123;456;7B8
Step 6: 123;456;78B
Visited: 7 | Time: 0.001 s

--- Genetic Algorithm ---
Step 0: 123;746;B58
Step 1: 123;746;758
Step 2: 123;B46;B58
Step 3: 132;746;B58
Step 4: B23;146;B58
Step 5: 123;B46;758
Step 6: B26;143;B58
Step 7: 123;546;B78
Step 8: B23;146;758
Step 9: 123;745;B68
Step 10: 183;B46;B52
Step 11: 183;B46;752
Step 12: 123;B56;B48
Visited: 13 | Time: 0.002 s
